In [13]:
import time
import asyncio
import contextlib
from pymongo import MongoClient
from motor.motor_asyncio import AsyncIOMotorClient

@contextlib.contextmanager
def timer(description):
    start_time = time.time()
    yield
    end_time = time.time()
    print('{} took {} seconds'.format(description, end_time - start_time))
    
sync_client = MongoClient().aiotest.testcol
aio_client = AsyncIOMotorClient(max_pool_size=10).aiotest.testcol
loop = asyncio.get_event_loop()

In [6]:
N = 1000

In [22]:
_ = sync_client.insert({'abc': i} for i in range(N))

In [25]:
with timer('sequential sync read'):
    for i in range(N):
        sync_client.find_one({'abc': i})

sequential sync read took 0.34887003898620605 seconds


In [9]:
with timer('sequential async read'):
    for i in range(N):
        loop.run_until_complete(aio_client.find_one({'abc': i}))

sequential async read took 0.5591599941253662 seconds


In [14]:
for chunk_size in range(1, 20):
    with timer('parallel async read with {} parallel requests'.format(chunk_size)):
        futures = [col.find_one({'abc': i})]
        for i in range(N // chunk_size):
            # interesting part:
            futures = [col.find_one({'abc': (i * chunk_size) + j})
                       for j in range(chunk_size)]
            loop.run_until_complete(asyncio.gather(*futures))

parallel async read with 1 parallel requests took 0.616163969039917 seconds
parallel async read with 2 parallel requests took 0.35162997245788574 seconds


parallel async read with 3 parallel requests took 0.2948617935180664 seconds


parallel async read with 4 parallel requests took 0.26516294479370117 seconds


parallel async read with 5 parallel requests took 0.24994611740112305 seconds


parallel async read with 6 parallel requests took 0.24745702743530273 seconds

...


parallel async read with 15 parallel requests took 0.22688078880310059 seconds


parallel async read with 16 parallel requests took 0.22031497955322266 seconds


parallel async read with 17 parallel requests took 0.22269797325134277 seconds


parallel async read with 18 parallel requests took 0.22491192817687988 seconds


parallel async read with 19 parallel requests took 0.21470904350280762 seconds